[Reference](https://towardsdatascience.com/an-approach-towards-convolutional-recurrent-neural-networks-a2e6ce722b19)

![rcnn](https://miro.medium.com/max/1400/1*etN2RhEkMJrEtJgWLvD9pQ.png)

In [ ]:
def get_model(data_in, data_out, _cnn_nb_filt, _cnn_pool_size, _rnn_nb, _fc_nb):
    spec_start = Input(shape=(data_in.shape[-3], data_in.shape[-2], data_in.shape[-1]))
    spec_x = spec_start
    for _i, _cnt in enumerate(_cnn_pool_size):
        spec_x = Conv2D(filters = cnn_nb_filt, kernel_size=(2, 2), padding='same')(spec_x)
        spec_x = BatchNormalization(axis=1)(spec_x)
        spec_x = Activation('relu')(spec_x)
        spec_x = MaxPooling2D(pool_size=(1, _cnn_pool_size[_i]))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)
        spec_x = Permute((2, 1, 3))(spec_x)
        spec_x = Reshape((data_in.shape[-2], -1))(spec_x)
    
    for _r in _rnn_nb:
        spec_x = Bidirectional(
        GRU(_r, activation='tanh', dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True),
        merge_mode='concat')(spec_x)
    
    for _f in _fc_nb:
        spec_x = TimeDistributed(Dense(_f))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)
    
    spec_x = TimeDistributed(Dense(data_out.shape[-1]))(spec_x)
    out = Activation('sigmoid', name='strong_out')(spec_x)
    
    _model = Model(inputs=spec_start, outputs=out)
    _model.compile(optimizer='Adam', loss='binary_crossentropy',metrics = ['accuracy'])
    _model.summary()

    return _model

In [ ]:
# Load model
# model = get_model(X, Y, cnn_nb_filt, cnn_pool_size, rnn_nb, fc_nb)